In [2]:
%%python

UsageError: %%python is a cell magic, but the cell body is empty.


In [3]:
sc.stop()

In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *
import numpy as np
import pandas as pd
sc = SparkContext.getOrCreate()
#
spark = SparkSession(sc)

In [5]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
#sc = SparkContext.getOrCreate()
#spark = SparkSession(sc)

In [6]:
loan_data = spark.read.csv("loan_data.csv", header="true", inferSchema="true")

In [7]:
#pants = pants.select("size","cost","color").filter(col("color") == "blue")

In [8]:
loan_data.show(5)

+--------+---------------+-----------+---+------------+
|have_job|marriage_status|year_income|age|allowed_loan|
+--------+---------------+-----------+---+------------+
|       1|              1|      39629| 45|           0|
|       0|              2|      24442| 27|           0|
|       0|              1|      59450| 33|           0|
|       0|              1|      43030| 27|           0|
|       0|              2|      36625| 36|           0|
+--------+---------------+-----------+---+------------+
only showing top 5 rows



STARTING ML

In [9]:
from pyspark.mllib.clustering import KMeans
from numpy import array
from math import sqrt

In [10]:
data = loan_data
data.show(5)

+--------+---------------+-----------+---+------------+
|have_job|marriage_status|year_income|age|allowed_loan|
+--------+---------------+-----------+---+------------+
|       1|              1|      39629| 45|           0|
|       0|              2|      24442| 27|           0|
|       0|              1|      59450| 33|           0|
|       0|              1|      43030| 27|           0|
|       0|              2|      36625| 36|           0|
+--------+---------------+-----------+---+------------+
only showing top 5 rows



Clustering

In [11]:
from pyspark.mllib.clustering import KMeans
from numpy import array
from math import sqrt
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler


Have a job: 1 = true, 0 = False <br>
marriage_status: 0 = divorsed, 1 = single, 2 = married <br>
year_income: amount in EU <br>
age: age of the person <br>

In [12]:
vectorAssembler = VectorAssembler(inputCols = ['have_job', 'marriage_status', 'year_income', 'age'], outputCol = 'features')
data = vectorAssembler.transform(data)
#data = data.select('features')


In [13]:
kmeans = KMeans(k= 4)
model = kmeans.fit(data)

In [14]:
# Make predictions
predictions = model.transform(data)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.7309206657160369


In [15]:
predictions.show(3)

+--------+---------------+-----------+---+------------+--------------------+----------+
|have_job|marriage_status|year_income|age|allowed_loan|            features|prediction|
+--------+---------------+-----------+---+------------+--------------------+----------+
|       1|              1|      39629| 45|           0|[1.0,1.0,39629.0,...|         2|
|       0|              2|      24442| 27|           0|[0.0,2.0,24442.0,...|         0|
|       0|              1|      59450| 33|           0|[0.0,1.0,59450.0,...|         2|
+--------+---------------+-----------+---+------------+--------------------+----------+
only showing top 3 rows



In [16]:
predictions.select('have_job','marriage_status','year_income','age','allowed_loan','prediction')\
.show(20)

+--------+---------------+-----------+---+------------+----------+
|have_job|marriage_status|year_income|age|allowed_loan|prediction|
+--------+---------------+-----------+---+------------+----------+
|       1|              1|      39629| 45|           0|         2|
|       0|              2|      24442| 27|           0|         0|
|       0|              1|      59450| 33|           0|         2|
|       0|              1|      43030| 27|           0|         2|
|       0|              2|      36625| 36|           0|         0|
|       1|              2|      51875| 55|           0|         2|
|       0|              1|      36783| 53|           0|         0|
|       1|              0|      54073| 35|           0|         2|
|       1|              0|      45205| 45|           0|         2|
|       0|              2|      50261| 48|           0|         2|
|       1|              2|      54325| 35|           0|         2|
|       1|              0|      34652| 44|           0|       